This notebook will show how to calculate differences between a DEM and ICESat2 points using the `reference_dem` class.

First, import `coregistration` and create the DEM object:

In [92]:
#import the module via an explicit path directly for now
import importlib.util
spec = importlib.util.spec_from_file_location("coregistration", "/home/jovyan/Assimilation/scripts/coregistration.py")
cor = importlib.util.module_from_spec(spec)
spec.loader.exec_module(cor)

In [93]:
dem_file_path = '/home/jovyan/data/reference_dem_clip.tif'

In [94]:
ds = cor.reference_dem(dem_file_path)
ds.calculate_bounding_box(4326)

Next, create a GeoDataFrame of ATL06 points. This will be done by our scripts later but is provided explicitly as an example for now. 

First download an example file using the scripts provided by Tian Li.

Then, create a GeoDataFrame from one of the files (e.g. the `gt2r` beam points on 2018-10-19):

In [95]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

example_file = '/home/jovyan/data/ATL06_Processed/processed_ATL06_20181019231110_03260102_003_01_gt2r.h5'
vnames = ['x', 'y', 'h_li']
i2_data = atl06_lib.read_h5(example_file, vnames)

points = []            
for p in range(np.shape(i2_data)[0]):
    points.append({
        'x': i2_data[p,0],
        'y': i2_data[p,1],
        'h': i2_data[p,2]})

df = pd.DataFrame.from_dict(points)

geometry = [Point(xyz) for xyz in zip(df['x'], df['y'], df['h'])]        
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=CRS('EPSG:'+str(ds.epsg)))

Next, the `colocate_icesat2_dem_points(icesat2_gdf)` can be called to find the difference between the provided GeoDataFrame of icesat2 points and the DEM points

In [96]:
ds.colocate_icesat2_dem_points(gdf)
print(ds.dem_is2_comparison)

                 x             y     dem_elev     is2_elev    dem-is2
1    586319.497387  5.395103e+06  1039.610107  1038.600952   1.009155
2    586317.188128  5.395124e+06  1054.310059  1039.262085  15.047974
3    586314.805116  5.395145e+06  1040.072998  1040.090332  -0.017334
4    586312.578000  5.395165e+06  1042.106812  1040.767700   1.339111
5    586310.466515  5.395184e+06  1041.342896  1041.149780   0.193115
..             ...           ...          ...          ...        ...
139  585518.317682  5.402210e+06  2480.963867  2485.448730  -4.484863
140  585516.129457  5.402230e+06  2482.430664  2486.523926  -4.093262
141  585513.934961  5.402250e+06  2484.899658  2488.912842  -4.013184
142  585486.745317  5.402489e+06  2606.960938  2607.850586  -0.889648
143  585484.533362  5.402509e+06  2611.922852  2613.419678  -1.496826

[143 rows x 5 columns]


Note that the comparison of points are stored in the `dem_reference` object